<a href="https://colab.research.google.com/github/TytanMikJas/DeepLearningJourney/blob/main/Gratuitously_complex_adding_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The most complex adding machine ever! (my first independent DL project)
The purpouse of my model is to predict the outcome of adding two numbers between -10 and 10. At first glance, the idea might sound idiotic, but my vision for the project is to allow my model to KNOW the answers, not calculate them. It's simmilar to how we solve 8+8. We do not actually calculate it we just... know it! :)

## Imports

In [97]:
# imports

# data and learning
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# plotting
import matplotlib.pyplot as plt
from IPython import display
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# measuring
import sklearn.metrics as skm

## Creating and processing data

In [119]:
# creating N number of 2 digit pairs (data) and their sum (labels)
def create_data(N: int):
    data = np.random.randint(-10, 10, size=(N, 2))
    labels = np.sum(data,axis=1)
    return data, labels

In [122]:
# INPUT NUMBER OF DATA TO GENERATE
data, labels = create_data(2500)

dataT = torch.tensor(data).float()
labelsT = torch.tensor(labels).long()

# dividing data into 80/10/10 sets (train, dev, test)
train_data, dev_test_data, train_labels, dev_test_labels = train_test_split(dataT, labelsT, train_size=0.8)
dev_data, test_data, dev_labels, test_labels = train_test_split(dev_test_data, dev_test_labels, train_size=0.5)

# converting data into TensorDatasets and then DataLoaders
train_data = TensorDataset(train_data, train_labels)
dev_data  = TensorDataset(dev_data, dev_labels)
test_data = TensorDataset(test_data, test_labels)

train_loader = DataLoader(train_data, batch_size=16,shuffle=True,drop_last=True)
dev_loader = DataLoader(dev_data, batch_size=dev_data.tensors[0].shape[0])
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

tensor([ 6, -1,  4,  ..., -1, -2, -3])


## Creating Deep Learning model

In [112]:
# function to create model, loss function and optimizer
def create_model():

    class AddingNet(nn.Module):

        # class initialisation
        def __init__(self):
            super().__init__()

            # input layer
            self.input = nn.Linear(2,16)

            # hidden layers
            self.hidden = nn.Linear(16, 8)

            # output layer
            self.output = nn.Linear(8, 39)

        # forward pass
        def forward(self, x):

            x = F.relu( self.input(x) )
            x = F.relu( self.hidden(x) )

            return self.output(x)

    # model instance
    net = AddingNet()

    # loss function
    lossfun = nn.CrossEntropyLoss()

    # optimizer
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, weight_decay=0.01)

    return net, lossfun, optimizer

## Trainining function

In [120]:
def function2trainTheModel():

    # number of epochs
    numepochs = 100

    # create a new model
    net, lossfun, optimizer = create_model()


    # initialize losses
    losses    = torch.zeros(numepochs)
    trainAcc  = []
    devAcc   = []

    # loop over epochs
    for epochi in range(numepochs):

        # loop over training data batches
        batchAcc  = []
        batchLoss = []
        for X,y in train_loader:
            print('1')
            # forward pass and loss
            yHat = net(X)
            print('1.5')
            loss = lossfun(yHat,y)
            print('2')

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('3')

            # loss from this batch
            batchLoss.append(loss.item())

            # compute accuracy
            matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
            matchesNumeric = matches.float()             # convert to numbers (0/1)
            accuracyPct = 100*torch.mean(matchesNumeric) # average and x100
            batchAcc.append( accuracyPct )               # add to list of accuracies
            print('4')


        # average accuracy and losses over batches
        trainAcc.append( np.mean(batchAcc) )
        losses[epochi] = np.mean(batchLoss)

        # test accuracy
        X,y = next(iter(dev_loader)) # extract X,y from test dataloader
        with torch.no_grad(): # deactivates autograd
            yHat = net(X)

        # compare the following really long line of code to the training accuracy lines
        devAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )


    return trainAcc, devAcc, losses, net

## Run the model

In [121]:
for i in range(10):

    # build and train the model
    trainAcc, devAcc, losses, net = function2trainTheModel()

    # visualization starts here
    fig,ax = plt.subplots(1,2,figsize=(12,5))

    ax[0].plot(losses)
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    ax[0].set_ylim([0,3])
    ax[0].set_title('Model loss')

    ax[1].plot(trainAcc,label='Train')
    ax[1].plot(devAcc,label='Test')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Accuracy (%)')
    ax[1].set_ylim([10,100])
    ax[1].set_title(f'Final model test accuracy: {devAcc[-1]:.2f}%')
    ax[1].legend()

    plt.show()

1
1.5


IndexError: ignored

JEBIE SIE TO ZE PREZIWDUJEMY od -20 do 20 i zapewne cross entrophy loss daje argmax no i przewiduje label -20, a nie ma takiego argmax